In [1]:
import lightgbm as lgb
from catboost import CatBoostRegressor
import os, glob
import json
import folium
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error
from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split



import warnings
# Filter out specific ValueWarnings from statsmodels
warnings.filterwarnings("ignore")

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# !ls /kaggle/input/

In [3]:
ndays=1
exps_dir = "../../exps"
if os.path.exists(exps_dir) == False: # tạo thư mục (nếu chưa có)
  os.makedirs(exps_dir, exist_ok=True)

save_dir = f"{exps_dir}/exp1"
os.makedirs(save_dir, exist_ok=True)

In [4]:
df_train_features=pd.read_csv(f'{save_dir}/train_lag{ndays}.csv')
df_train_features

,row_id,county,is_business,product_type,is_consumption,prediction_unit_id,date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,...,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),target
0,366048,0,0,1,0,0,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.000000,1.000000,0.000
1,366049,0,0,1,1,0,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_1_1,0.017166,0.999853,0.000000,1.000000,442.226
2,366050,0,0,2,0,1,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_2_0,0.017166,0.999853,0.000000,1.000000,0.000
3,366051,0,0,2,1,1,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_2_1,0.017166,0.999853,0.000000,1.000000,44.899
4,366052,0,0,3,0,2,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_3_0,0.017166,0.999853,0.000000,1.000000,0.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651897,2018347,15,1,0,1,64,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_0_1,0.522133,-0.852864,-0.258819,0.965926,197.233
1651898,2018348,15,1,1,0,59,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_1_0,0.522133,-0.852864,-0.258819,0.965926,0.000
1651899,2018349,15,1,1,1,59,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_1_1,0.522133,-0.852864,-0.258819,0.965926,28.404
1651900,2018350,15,1,3,0,60,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_3_0,0.522133,-0.852864,-0.258819,0.965926,0.000


In [5]:
df_train_features.drop(columns=['row_id'])

,county,is_business,product_type,is_consumption,prediction_unit_id,date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id,...,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),target
0,0,0,1,0,0,2022-01-01,80.0,94.0,2021-12-30,122.0,...,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.000000,1.000000,0.000
1,0,0,1,1,0,2022-01-01,80.0,94.0,2021-12-30,122.0,...,1,6,1,2022,0_0_1_1,0.017166,0.999853,0.000000,1.000000,442.226
2,0,0,2,0,1,2022-01-01,80.0,94.0,2021-12-30,122.0,...,1,6,1,2022,0_0_2_0,0.017166,0.999853,0.000000,1.000000,0.000
3,0,0,2,1,1,2022-01-01,80.0,94.0,2021-12-30,122.0,...,1,6,1,2022,0_0_2_1,0.017166,0.999853,0.000000,1.000000,44.899
4,0,0,3,0,2,2022-01-01,80.0,94.0,2021-12-30,122.0,...,1,6,1,2022,0_0_3_0,0.017166,0.999853,0.000000,1.000000,0.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651897,15,1,0,1,64,2023-05-31,29.0,34.0,2023-05-29,637.0,...,31,3,5,2023,15_1_0_1,0.522133,-0.852864,-0.258819,0.965926,197.233
1651898,15,1,1,0,59,2023-05-31,29.0,34.0,2023-05-29,637.0,...,31,3,5,2023,15_1_1_0,0.522133,-0.852864,-0.258819,0.965926,0.000
1651899,15,1,1,1,59,2023-05-31,29.0,34.0,2023-05-29,637.0,...,31,3,5,2023,15_1_1_1,0.522133,-0.852864,-0.258819,0.965926,28.404
1651900,15,1,3,0,60,2023-05-31,29.0,34.0,2023-05-29,637.0,...,31,3,5,2023,15_1_3_0,0.522133,-0.852864,-0.258819,0.965926,0.000


In [18]:
df_train_features['target_1']

0            0.004
1          412.153
2            0.000
3           43.871
4            0.610
            ...   
1651897    188.167
1651898      0.000
1651899     31.484
1651900      0.000
1651901    177.056
Name: target_1, Length: 1651902, dtype: float64

In [7]:
numeric_columns=(list(df_train_features.select_dtypes('number')))[0:len(list(df_train_features.select_dtypes('number')))-10]
numeric_columns.append('target')
numeric_columns.append('target_1')
numeric_columns.remove('row_id')
numeric_columns.remove('data_block_id')
numeric_columns.remove('data_block_id_right')
numeric_columns.remove('hours_ahead')
numeric_columns.remove('hours_ahead_fl_7d')
numeric_columns.remove('county')
numeric_columns.remove('product_type')
numeric_columns.remove('is_consumption')
numeric_columns.remove('prediction_unit_id')
numeric_columns.remove('is_business')


numeric_columns

['lowest_price_per_mwh',
 'highest_price_per_mwh',
 'eic_count',
 'installed_capacity',
 'euros_per_mwh',
 'temperature',
 'dewpoint',
 'cloudcover_high',
 'cloudcover_low',
 'cloudcover_mid',
 'cloudcover_total',
 '10_metre_u_wind_component',
 '10_metre_v_wind_component',
 'direct_solar_radiation',
 'surface_solar_radiation_downwards',
 'snowfall',
 'total_precipitation',
 'hours_ahead_fl',
 'temperature_fl',
 'dewpoint_fl',
 'cloudcover_high_fl',
 'cloudcover_low_fl',
 'cloudcover_mid_fl',
 'cloudcover_total_fl',
 '10_metre_u_wind_component_fl',
 '10_metre_v_wind_component_fl',
 'direct_solar_radiation_fl',
 'surface_solar_radiation_downwards_fl',
 'snowfall_fl',
 'total_precipitation_fl',
 'hours_ahead_fd_7d',
 'temperature_fd_7d',
 'dewpoint_fd_7d',
 'cloudcover_high_fd_7d',
 'cloudcover_low_fd_7d',
 'cloudcover_mid_fd_7d',
 'cloudcover_total_fd_7d',
 '10_metre_u_wind_component_fd_7d',
 '10_metre_v_wind_component_fd_7d',
 'direct_solar_radiation_fd_7d',
 'surface_solar_radiation_do

In [8]:
category_columns=list(df_train_features.select_dtypes('category'))
category_columns.append('data_block_id')
category_columns.append('data_block_id_right')
category_columns.append('hours_ahead')
category_columns.append('hours_ahead_fl_7d')
category_columns.append('county')
category_columns.append('row_id')
category_columns.append('product_type')
category_columns.append('is_consumption')
category_columns.append('prediction_unit_id')
category_columns.append('segment')
category_columns.append('is_business')

In [9]:
list(df_train_features.select_dtypes('object'))

['date', 'origin_date', 'origin_date_right', 'segment']

In [20]:
import pandas as pd

def aggregate_hourly_to_daily(df, numeric_columns):
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].dt.date
    df['origin_date'] = pd.to_datetime(df['origin_date'])
    df['origin_date'] = df['origin_date'].dt.date
    df['origin_date_right'] = pd.to_datetime(df['origin_date_right'])
    df['origin_date_right'] = df['origin_date_right'].dt.date

    grouped = df.groupby([
        # 'customer_id', 
        'county',
        'is_business', 
        'product_type', 
        'is_consumption', 
        'prediction_unit_id', 
        'date', 
        'data_block_id', 
        'data_block_id_right', 
        # 'hours_ahead', 
        # 'hours_ahead_fl_7d',
        'origin_date',
        'origin_date_right',
        'segment',
        "sin(dayofyear)",
        'cos(dayofyear)',
    ])

    agg_dict = {
        'target': 'sum',
        'target_1':'sum',
        'lowest_price_per_mwh': 'min',
        'highest_price_per_mwh': 'max'
    }
    
    for col in numeric_columns[2:-1]:
        agg_dict[col] = 'mean'

    result = grouped.agg(agg_dict).reset_index()

    result = result.sort_values(by=['date','county','is_business', 'product_type'])
    
    return result




In [21]:
df = aggregate_hourly_to_daily(df_train_features, numeric_columns)
df

,county,is_business,product_type,is_consumption,prediction_unit_id,date,data_block_id,data_block_id_right,origin_date,origin_date_right,...,surface_pressure_hl_7d,cloudcover_total_hl_7d,cloudcover_low_hl_7d,cloudcover_mid_hl_7d,cloudcover_high_hl_7d,windspeed_10m_hl_7d,winddirection_10m_hl_7d,shortwave_radiation_hl_7d,direct_solar_radiation_hl_7d,diffuse_radiation_hl_7d
0,0,0,1,0,0,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,993.287497,71.611111,63.798611,32.465278,29.888889,4.349151,252.034723,11.583333,3.208333,8.375000
516,0,0,1,1,0,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,993.287497,71.611111,63.798611,32.465278,29.888889,4.349151,252.034723,11.583333,3.208333,8.375000
1032,0,0,2,0,1,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,993.287497,71.611111,63.798611,32.465278,29.888889,4.349151,252.034723,11.583333,3.208333,8.375000
1548,0,0,2,1,1,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,993.287497,71.611111,63.798611,32.465278,29.888889,4.349151,252.034723,11.583333,3.208333,8.375000
2064,0,0,3,0,2,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,993.287497,71.611111,63.798611,32.465278,29.888889,4.349151,252.034723,11.583333,3.208333,8.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66781,15,1,0,1,64,2023-05-31,637.0,637.0,2023-05-29,2023-05-29,...,1004.549091,58.109091,4.636364,74.890909,30.036364,2.254040,247.036364,139.963636,66.909091,73.054545
67297,15,1,1,0,59,2023-05-31,637.0,637.0,2023-05-29,2023-05-29,...,1004.549091,58.109091,4.636364,74.890909,30.036364,2.254040,247.036364,139.963636,66.909091,73.054545
67813,15,1,1,1,59,2023-05-31,637.0,637.0,2023-05-29,2023-05-29,...,1004.549091,58.109091,4.636364,74.890909,30.036364,2.254040,247.036364,139.963636,66.909091,73.054545
68329,15,1,3,0,60,2023-05-31,637.0,637.0,2023-05-29,2023-05-29,...,1004.549091,58.109091,4.636364,74.890909,30.036364,2.254040,247.036364,139.963636,66.909091,73.054545


In [22]:
df.target_1

0            8.457
516      10455.061
1032         0.000
1548      1056.084
2064        26.209
           ...    
66781    11490.608
67297     2889.724
67813      810.093
68329     6876.425
68845     7462.889
Name: target_1, Length: 68846, dtype: float64

In [23]:
# for 'segment' in category_columns:
label_encoder = LabelEncoder()
df['segment'] = label_encoder.fit_transform(df['segment'])
#     df_train_features['segment'] = label_encoder  

In [24]:
df=df.drop(columns=['date','origin_date','origin_date_right'])
# df.isnull().sum()
df.dropna(inplace=True)
len(df)

67714

In [25]:
Y=df['target']
X=df.drop('target',axis=1)

In [26]:
df_train,df_valid = train_test_split(df, test_size=0.33, random_state=42, shuffle=False)
print(df_train.shape, df_valid.shape)

(45368, 95) (22346, 95)


In [27]:
Y_train = df_train['target']
Y_valid = df_valid['target']
print(Y_train.shape, Y_valid.shape)

(45368,) (22346,)


In [28]:
scaler = StandardScaler()
df_train[numeric_columns] = scaler.fit_transform(df_train[numeric_columns])
df_train

,county,is_business,product_type,is_consumption,prediction_unit_id,data_block_id,data_block_id_right,segment,sin(dayofyear),cos(dayofyear),...,surface_pressure_hl_7d,cloudcover_total_hl_7d,cloudcover_low_hl_7d,cloudcover_mid_hl_7d,cloudcover_high_hl_7d,windspeed_10m_hl_7d,winddirection_10m_hl_7d,shortwave_radiation_hl_7d,direct_solar_radiation_hl_7d,diffuse_radiation_hl_7d
0,0,0,1,0,0,122.0,122.0,0,0.017166,0.999853,...,-1.134587,0.436670,0.635065,-0.055793,-0.169589,0.197806,0.787020,-1.194832,-0.996038,-1.342601
516,0,0,1,1,0,122.0,122.0,1,0.017166,0.999853,...,-1.134587,0.436670,0.635065,-0.055793,-0.169589,0.197806,0.787020,-1.194832,-0.996038,-1.342601
1032,0,0,2,0,1,122.0,122.0,2,0.017166,0.999853,...,-1.134587,0.436670,0.635065,-0.055793,-0.169589,0.197806,0.787020,-1.194832,-0.996038,-1.342601
1548,0,0,2,1,1,122.0,122.0,3,0.017166,0.999853,...,-1.134587,0.436670,0.635065,-0.055793,-0.169589,0.197806,0.787020,-1.194832,-0.996038,-1.342601
2064,0,0,3,0,2,122.0,122.0,4,0.017166,0.999853,...,-1.134587,0.436670,0.635065,-0.055793,-0.169589,0.197806,0.787020,-1.194832,-0.996038,-1.342601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51268,11,1,0,1,67,466.0,466.0,31,-0.352752,0.935717,...,-0.216674,1.084762,1.458335,-0.571724,-0.963633,-0.553104,1.236786,-1.202156,-1.031642,-1.280201
51784,11,1,1,0,46,466.0,466.0,32,-0.352752,0.935717,...,-0.216674,1.084762,1.458335,-0.571724,-0.963633,-0.553104,1.236786,-1.202156,-1.031642,-1.280201
52300,11,1,1,1,46,466.0,466.0,33,-0.352752,0.935717,...,-0.216674,1.084762,1.458335,-0.571724,-0.963633,-0.553104,1.236786,-1.202156,-1.031642,-1.280201
53572,11,1,3,0,48,466.0,466.0,36,-0.352752,0.935717,...,-0.216674,1.084762,1.458335,-0.571724,-0.963633,-0.553104,1.236786,-1.202156,-1.031642,-1.280201


In [ ]:
df_valid[numeric_columns] = scaler.fit_transform(df_valid[numeric_columns])
df_valid

,county,is_business,product_type,is_consumption,prediction_unit_id,data_block_id,data_block_id_right,segment,sin(dayofyear),cos(dayofyear),...,surface_pressure_hl_7d,cloudcover_total_hl_7d,cloudcover_low_hl_7d,cloudcover_mid_hl_7d,cloudcover_high_hl_7d,windspeed_10m_hl_7d,winddirection_10m_hl_7d,shortwave_radiation_hl_7d,direct_solar_radiation_hl_7d,diffuse_radiation_hl_7d
55636,13,0,1,0,50,466.0,466.0,40,-0.352752,0.935717,...,-0.028652,0.834884,1.211288,-0.947556,-1.166219,-0.781083,1.268545,-0.922324,-0.774748,-1.040202
56152,13,0,1,1,50,466.0,466.0,41,-0.352752,0.935717,...,-0.028652,0.834884,1.211288,-0.947556,-1.166219,-0.781083,1.268545,-0.922324,-0.774748,-1.040202
56668,13,0,3,0,51,466.0,466.0,42,-0.352752,0.935717,...,-0.028652,0.834884,1.211288,-0.947556,-1.166219,-0.781083,1.268545,-0.922324,-0.774748,-1.040202
57184,13,0,3,1,51,466.0,466.0,43,-0.352752,0.935717,...,-0.028652,0.834884,1.211288,-0.947556,-1.166219,-0.781083,1.268545,-0.922324,-0.774748,-1.040202
57700,13,1,1,0,63,466.0,466.0,44,-0.352752,0.935717,...,-0.028652,0.834884,1.211288,-0.947556,-1.166219,-0.781083,1.268545,-0.922324,-0.774748,-1.040202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66781,15,1,0,1,64,637.0,637.0,63,0.522133,-0.852864,...,-0.168007,-0.184181,-1.237774,1.330452,-0.313649,-1.183988,0.657357,0.393930,0.078553,1.120285
67297,15,1,1,0,59,637.0,637.0,64,0.522133,-0.852864,...,-0.168007,-0.184181,-1.237774,1.330452,-0.313649,-1.183988,0.657357,0.393930,0.078553,1.120285
67813,15,1,1,1,59,637.0,637.0,65,0.522133,-0.852864,...,-0.168007,-0.184181,-1.237774,1.330452,-0.313649,-1.183988,0.657357,0.393930,0.078553,1.120285
68329,15,1,3,0,60,637.0,637.0,66,0.522133,-0.852864,...,-0.168007,-0.184181,-1.237774,1.330452,-0.313649,-1.183988,0.657357,0.393930,0.078553,1.120285


In [19]:
# df_train.drop('row_id',axis=1)
# df_valid.drop('row_id',axis=1)

In [20]:
df.to_csv(f'{save_dir}/train_lag{ndays}_days_pre_processing.csv')
df_train.to_csv(f'{save_dir}/df_train{ndays}_days_pre_processing.csv')
df_valid.to_csv(f'{save_dir}/df_valid{ndays}_days_pre_processing.csv')

In [21]:
Y_train.to_csv(f'{save_dir}/target_train{ndays}_pre_processing.csv')
Y_valid.to_csv(f'{save_dir}/target_valid{ndays}_pre_processing.csv')
X.to_csv(f'{save_dir}/X_train_lag{ndays}_days_pre_processing.csv')
Y.to_csv(f'{save_dir}/y_train_lag{ndays}_days_pre_processing.csv')

In [22]:
from joblib import dump
scaler=StandardScaler()
scaler.fit(Y.values.reshape(-1,1))
dump(scaler,f'{save_dir}/scaler_y.pkl')


['../../exps/exp1/scaler_y.pkl']